In [29]:
## Imports
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
from datetime import datetime
import os
import urllib.request

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

try:
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
except Exception as e:
    os.system("python -m spacy download en")
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-pack

In [30]:
def download_project_maslow_files():
    """The purpose of this function is to handle getting the nonprofit.txt and nonprofit_text.txt
    files.  This functin will create a data directory, add a .gitignore file so
    source control does not pick up the txt files, and then check to see
    if either nonprofit.txt and nonprofit_text.txt have been downloaded yet.
    If not, it will proceed to download the files.
    """

    if os.path.isdir("data") is False:
        print("data/ folder does not exist, creating it now")
        os.mkdir("data")

    if os.path.isfile("data/.gitignore") is False:
        os.system("echo *.txt > data/.gitignore")

    if os.path.isfile("data/nonprofit.txt") is False:
        print("Downloading nonprofit.txt...")
        urllib.request.urlretrieve("https://mtsu-dsi-hackathon-2022.s3.amazonaws.com/nonprofit.txt", 'data/nonprofit.txt')

    if os.path.isfile("data/nonprofit_text.txt") is False:
        print("Downloading nonprofit_text.txt...")
        urllib.request.urlretrieve("https://mtsu-dsi-hackathon-2022.s3.amazonaws.com/nonprofit_text.txt", 'data/nonprofit_text.txt')

In [31]:
def get_non_profit_df():
    """This function will first run the download_project_maslow_files() to check to
    see if the files have been downloaded to the `data/` folder.  If not,
    it will download the files.
    Then, it will read in the file and return a dataframe with the nonprofit
    """

    download_project_maslow_files()

    col_types = {
        "nonprofit_id": "Int64",
        "reporting_year": "Int64",
        "ein": "Int64",
        "businessname": "str",
        "phone": "str",
        "address1": "str",
        "address2": "str",
        "city": "str",
        "stabbrv": "str",
        "zip": "str"
      }

    df = pd.read_csv("data/nonprofit.txt", sep = "|", dtype=col_types)

    return df
df_np = get_non_profit_df()
df_np = df_np[["nonprofit_id", "stabbrv"]]


In [53]:
def get_non_profit_text_df():
    """This function will first run the download_project_maslow_files() to check to
    see if the files have been downloaded to the `data/` folder.  If not,
    it will download the files.
    Then, it will read in the file and return a dataframe with the nonprofit_text
    """

    download_project_maslow_files()

    col_types = {
        "nonprofit_text_id": "Int64",
        "reporting_year": "Int64",
        "nonprofit_id": "Int64",
        "grouptype": "str",
        "description": "str"
    }

    df = pd.read_csv("data/nonprofit_text.txt", sep = "|", encoding="cp1252", dtype=col_types)

    return df

non_agg_df = get_non_profit_text_df()

# get rid of descriptions that are empty
filtered_df = non_agg_df[non_agg_df["description"].str.len() > 0]

df_text = filtered_df.groupby(['nonprofit_id'], as_index = False).agg({'description': ' '.join})

# filter down to TN
df_text = df_text.merge(df_np, on="nonprofit_id")
df_text = df_text[df_text["stabbrv"] == "TN"]

# filter down to aggregated descriptions greater than 10
df_text = df_text[df_text["description"].str.split().str.len() > 10]

# reset dataframe index after filtering
df_text = df_text.reset_index()

In [33]:
len(df_text["nonprofit_id"].unique())

6074

In [34]:
df_text.shape

(6074, 3)

In [35]:
# create a list of the description
data = df_text.description.values.tolist()
np.shape(data)

(6074,)

In [36]:
# tokenize and clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence).encode('utf-8'), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
np.shape(data_words)

/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(6074,)

In [37]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['provide', 'sound', 'supervised', 'provide', 'sound', 'supervised', 'recreational', 'sports', 'program', 'for', 'youth', 'of', 'our', 'community', 'in', 'which', 'they', 'receive', 'sports', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball', 'program', 'powell', 'recreation', 'commission', 'inc']


In [38]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if(len(token) > 3)])
    return texts_out

In [39]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

data_words_nostops[0]

['provide',
 'sound',
 'supervised',
 'provide',
 'sound',
 'supervised',
 'recreational',
 'sports',
 'program',
 'youth',
 'community',
 'receive',
 'sports',
 'training',
 'beneficial',
 'healthy',
 'recreation',
 'football',
 'program',
 'baseball',
 'program',
 'powell',
 'recreation',
 'commission',
 'inc']

In [40]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_bigrams[0]

['provide',
 'sound',
 'supervised',
 'provide',
 'sound',
 'supervised',
 'recreational',
 'sports',
 'program',
 'youth',
 'community',
 'receive',
 'sports',
 'training',
 'beneficial',
 'healthy',
 'recreation',
 'football',
 'program',
 'baseball',
 'program',
 'powell',
 'recreation',
 'commission',
 'inc']

In [41]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops)

data_lemmatized[0]

['provide',
 'sound',
 'supervised',
 'provide',
 'sound',
 'supervised',
 'recreational',
 'sport',
 'program',
 'youth',
 'community',
 'receive',
 'sport',
 'training',
 'beneficial',
 'healthy',
 'recreation',
 'football',
 'program',
 'baseball',
 'program',
 'powell',
 'recreation',
 'commission']

In [42]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 2), (9, 1), (10, 2), (11, 1), (12, 2), (13, 2), (14, 2), (15, 1), (16, 1)]]


In [43]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [59]:
def get_readable_topic(input_doc):
    if len(input_doc) > 1:
        doc = lda_model.id2word.doc2bow(input_doc)
        doc_topics, word_topics, phi_values = lda_model.get_document_topics(doc, per_word_topics=True)
        return lda_model.id2word[doc_topics[0][0]]
    else:
        return None

index = 0
for doc in data_lemmatized:
    df_text.loc[index,"topic_result"] = get_readable_topic(doc)
    index = index + 1

In [61]:
df_text["topic_result"].unique()

array(['baseball', 'community', 'beneficial', 'commission', 'powell',
       'football', 'healthy'], dtype=object)

In [62]:
# export model results into unique file
df_for_csv = df_text[["nonprofit_id", "topic_result", "description"]]
df_for_csv.to_csv(f"{datetime.now().strftime('%Y_%m_%d_%H_%M')}_unique_count_{len(df_text['nonprofit_id'].unique())}_nonprofit_text_output.csv", index=False, sep="|")

In [45]:
len(df_text["nonprofit_id"].unique())

6075

In [46]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.423754226267624

Coherence Score:  0.39342510289550053


In [47]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/aantonison/L

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.230890 -0.089768       1        1  23.083518
11    -0.145285  0.137976       2        1  16.902825
9     -0.063151 -0.245246       3        1  12.124882
3     -0.111835 -0.076359       4        1   9.502251
0     -0.158817  0.152930       5        1   8.193211
6      0.022966 -0.140979       6        1   7.815048
10     0.007439 -0.134964       7        1   6.215697
5     -0.096503  0.257638       8        1   6.072438
7      0.161915 -0.135251       9        1   3.841487
4      0.165012  0.137132      10        1   2.712407
2      0.195296  0.116037      11        1   2.669224
1      0.253853  0.020854      12        1   0.867010, topic_info=           Term          Freq         Total Category  logprob  loglift
99      service   5130.000000   5130.000000  Default  30.0000  30.0000
8       provide  10247.000000  10247.000000  Default  29.0000  29.0000
333      school   3138.000000   3138.000000  Default  28.0000  28.0000
177   tennessee   3738.000000   3738.000000  Default  27.0000  27.0000
148      health   2783.000000   2783.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
3272       casa     28.414759     29.418242  Topic12  -4.7914   4.7132
1649       hear     26.641518     27.645012  Topic12  -4.8559   4.7109
3802     orient     26.586622     27.590113  Topic12  -4.8579   4.7108
574     freedom     24.749416     25.752902  Topic12  -4.9295   4.7081
2528      excel     23.245166     24.248644  Topic12  -4.9922   4.7056

[557 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
489      10  0.983695   abandon
1369      7  0.998245     abuse
386       4  0.998202  academic
62        3  0.386085   academy
62        9  0.607240   academy
...     ...       ...       ...
282       3  0.557036      year
282       4  0.240573      year
282       5  0.036427      year
373       1  0.998214     young
16        1  0.999496     youth

[701 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 12, 10, 4, 1, 7, 11, 6, 8, 5, 3, 2])